In [ ]:
import os

if os.getenv("COLAB_RELEASE_TAG"):
    try:
        import chemprop
    except ImportError:
        !git clone https://github.com/chemprop/chemprop.git
        %cd chemprop
        !pip install .
        %cd examples


Cloning into 'chemprop'...
remote: Enumerating objects: 24846, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 24846 (delta 58), reused 19 (delta 19), pack-reused 24744 (from 5)
Receiving objects: 100% (24846/24846), 817.45 MiB | 44.01 MiB/s, done.
Resolving deltas: 100% (17856/17856), done.
/kaggle/working/chemprop
Processing /kaggle/working/chemprop
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 15.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━ 341.4/664.8 MB 185.0 MB/s eta 0:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.4 MB/s eta 0:00:00:00:0100

In [4]:
pip install chemprop rdkit optuna 

  Using cached rdkit-2024.9.6-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (4.0 kB)
  Using cached lightning-2.5.1-py3-none-any.whl.metadata (39 kB)
  Using cached astartes-1.3.0-py3-none-any.whl.metadata (26 kB)
  Using cached ConfigArgParse-1.7-py3-none-any.whl.metadata (23 kB)
  Using cached descriptastorus-2.8.0-py3-none-any.whl.metadata (364 bytes)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-manylinux20

In [ ]:
from rdkit import Chem, RDLogger
import pandas as pd
from tqdm.auto import tqdm

RDLogger.DisableLog('rdApp.*')

test_data = pd.read_csv('/kaggle/input/sibur-data/sibur_element_119_final_test_data80.csv')
train_data = pd.read_csv('/kaggle/input/sibur-data/sibur_element_119_final_train_data80.csv')

def is_valid_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return mol is not None

tqdm.pandas() 

train_data = train_data[train_data['SMILES'].progress_apply(is_valid_smiles)]
test_data = test_data[test_data['SMILES'].progress_apply(is_valid_smiles)]

train_data.to_csv('/kaggle/working/train_clean.csv', index=False)
test_data.to_csv('/kaggle/working/test_clean.csv', index=False)

  0%|          | 0/13403 [00:00<?, ?it/s]

  0%|          | 0/2630 [00:00<?, ?it/s]

ПОДБООР ГИПЕРПАРАМЕТРОВ


In [ ]:
!pip install --upgrade chemprop

In [ ]:
!chemprop hpopt \
  --data-path /kaggle/working/train_clean.csv \
  --task-type regression \
  --smiles-columns SMILES \
  --target-columns LogP \
  --search-parameter-keywords all \
  --hpopt-save-dir /kaggle/working/hpopt_results_1fold \
  --split-type SCAFFOLD BALANCED \
  --raytune-num-samples 3 \
  --raytune-search-algorithm optuna \
  --num-workers 4 \
  --raytune-use-gpu \
  --raytune-num-cpus 4 \
  --raytune-num-gpus 2 \
  --logfile /kaggle/working/hpopt_log_1fold.txt \
  --num-replicates 1 \
  --epochs 10 \
  --patience 3 \
  --raytune-grace-period 3

usage: chemprop hpopt [-h] [--logfile [LOGFILE]] [-v] [-q]
                      [-s SMILES_COLUMNS [SMILES_COLUMNS ...]]
                      [-r REACTION_COLUMNS [REACTION_COLUMNS ...]] [--no-header-row]
                      [-n NUM_WORKERS] [-b BATCH_SIZE] [--accelerator ACCELERATOR]
                      [--devices DEVICES]
                      [--rxn-mode {REAC_PROD,REAC_PROD_BALANCE,REAC_DIFF,REAC_DIFF_BALANCE,PROD_DIFF,PROD_DIFF_BALANCE}]
                      [--multi-hot-atom-featurizer-mode {V1,V2,ORGANIC,RIGR}] [--keep-h] [--add-h]
                      [--molecule-featurizers {morgan_binary,morgan_count,rdkit_2d,v1_rdkit_2d,v1_rdkit_2d_normalized,charge} [{morgan_binary,morgan_count,rdkit_2d,v1_rdkit_2d,v1_rdkit_2d_normalized,charge} ...]]
                      [--descriptors-path DESCRIPTORS_PATH] [--no-descriptor-scaling]
                      [--no-atom-feature-scaling] [--no-atom-descriptor-scaling]
                      [--no-bond-feature-scaling]
                  

In [ ]:
import os
import json

ray_results_dir = '/kaggle/working/hpopt_results/ray_results'
data_path = '/kaggle/working/train_clean.csv'
save_dir = '/kaggle/working/final_model'

best_loss = float('inf')
best_params = {}
best_trial_dir = ''

for trial_name in os.listdir(ray_results_dir):
    trial_path = os.path.join(ray_results_dir, trial_name)
    result_file = os.path.join(trial_path, 'result.json')
    params_file = os.path.join(trial_path, 'params.json')

    if os.path.exists(result_file) and os.path.exists(params_file):
        with open(result_file, 'r') as f:
            result = json.load(f)
            loss = result.get('validation_loss', result.get('val_loss', float('inf')))

        if loss < best_loss:
            best_loss = loss
            best_trial_dir = trial_path

with open(os.path.join(best_trial_dir, 'params.json')) as f:
    best_params = json.load(f)

cmd = [
    'chemprop', 'train',
    '--data-path', data_path,
    '--task-type', 'regression',
    '--smiles-columns', 'SMILES',
    '--target-columns', 'LogP',
    '--split-type', 'scaffold_balanced',
    '--save-dir', save_dir,
    '--gpu'
]

for key, value in best_params.items():
    cmd.append(f'--{key.replace("_", "-")}')
    cmd.append(str(value))

print("\n💡 Скопируй и запусти эту команду:\n")
print(" ".join(cmd))

In [ ]:
model_paths = [
    f"/kaggle/working/model_{i}/best.pt" for i in range(64)]

In [ ]:
import subprocess 

message_hidden_dim = 300  # Размер скрытого слоя в MPNN (рекомендуемые значения: 300, 600, 1200)
depth = 6  # Количество шагов передачи сообщений (рекомендуемые значения: 3, 4, 5)
ffn_hidden_dim = 1000  # Размер скрытого слоя в FFN (рекомендуемые значения: 300, 600)
ffn_num_layers = 1  # Количество слоёв в FFN (рекомендуемые значения: 1, 2, 3)
dropout = 0.05  # Вероятность дропаута для регуляризации (рекомендуемые значения: 0.0, 0.1, 0.2)
activation = "RELU"  # Функция активации (варианты: RELU, LEAKYRELU, PRELU, TANH, SELU, ELU)

# Параметры обучения
epochs = 200  # Количество эпох (рекомендуемое: 100-300)
patience = 10  # Эпохи без улучшения для ранней остановки (рекомендуемое: 10-30)
batch_size = 32  # Размер пакета (рекомендуемые значения: 32, 64, 128)
max_lr = 0.000269246231109958  # Максимальная скорость обучения (рекомендуемое: 0.0005-0.002)
init_lr_ratio = 0.17336685463715054  # Начальная скорость обучения (рекомендуемое: 0.00005-0.0002)
final_lr_ratio = 0.04548620831291529  # Конечная скорость обучения (рекомендуемое: 0.00005-0.0002)
warmup_epochs = 3   # Эпохи разогрева от init_lr до max_lr (рекомендуемое: 2-5)
split_type = "RANDOM"  # Тип разделения данных (варианты: RANDOM, SCAFFOLD_BALANCED)
ensemble_size = 1 # Количество моделей в ансамбле (рекомендуемое: 3-7)
aggregation = 'sum'
aggregation_norm = 74

# Пути к пользовательским дескрипторам (опционально)
train_descriptors_path = None  # Путь к CSV с дескрипторами для обучения (например, "train_descriptors.csv")
test_descriptors_path = None  # Путь к CSV с дескрипторами для теста (например, "test_descriptors.csv")

train_command = [
    "chemprop",
    "train",
    "--data-path", "/kaggle/working/train_clean.csv",   
    "--task-type", "regression",
    "--output-dir", "/kaggle/working/model_best",
    "--smiles-columns", "SMILES",
    "--target-columns", "LogP",
    "--epochs", str(epochs),
    "--patience", str(patience),
    "--split", split_type,
    "--ensemble-size", str(ensemble_size),
    "--metric", "rmse",
    "--message-hidden-dim", str(message_hidden_dim),
    "--depth", str(depth),
    "--ffn-hidden-dim", str(ffn_hidden_dim),
    "--ffn-num-layers", str(ffn_num_layers),
    "--dropout", str(dropout),
    "--activation", activation,
    "--batch-size", str(batch_size),
    "--max-lr", str(max_lr),
    "--init_lr_ratio", str(init_lr_ratio),
    "--final_lr_ratio", str(final_lr_ratio),
    "--num-workers", '4',
    "--warmup-epochs", str(warmup_epochs),
    '--logfile', '/kaggle/working/log_train_best.txt',
    '--aggregation', str(aggregation), 
    '--aggregation-norm', str(aggregation_norm)
]

subprocess.run(train_command)

usage: chemprop [-h] {train,predict,convert,fingerprint,hpopt} ...
chemprop: error: unrecognized arguments: --init_lr_ratio 0.17336685463715054 --final_lr_ratio 0.04548620831291529


CompletedProcess(args=['chemprop', 'train', '--data-path', '/kaggle/working/train_clean.csv', '--task-type', 'regression', '--output-dir', '/kaggle/working/model_best', '--smiles-columns', 'SMILES', '--target-columns', 'LogP', '--epochs', '200', '--patience', '10', '--split', 'RANDOM', '--ensemble-size', '1', '--metric', 'rmse', '--message-hidden-dim', '300', '--depth', '6', '--ffn-hidden-dim', '1000', '--ffn-num-layers', '1', '--dropout', '0.05', '--activation', 'RELU', '--batch-size', '32', '--max-lr', '0.000269246231109958', '--init_lr_ratio', '0.17336685463715054', '--final_lr_ratio', '0.04548620831291529', '--num-workers', '4', '--warmup-epochs', '3', '--logfile', '/kaggle/working/log_train_best.txt', '--aggregation', 'sum', '--aggregation-norm', '74'], returncode=2)

In [ ]:
import subprocess

train_command_best = [
    "chemprop",
    "train",
    "--data-path", "/kaggle/working/train_clean.csv",   
    "--task-type", "regression",
    "--output-dir", "/kaggle/working/models_best2",
    "--smiles-columns", "SMILES",
    "--target-columns", "LogP",
    "--epochs", '103',
    '--ensemble-size', '32',
    '--patience', '10',
    '--config-path', '/kaggle/input/best-nn-conf/best_config.toml']

subprocess.run(train_command_best)


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO: Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/2
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow 

Wrote config file to /kaggle/working/models_best2/config.toml
Wrote config file to /kaggle/working/models_best2/config.toml


INFO: 
  | Name            | Type               | Params | Mode 
---------------------------------------------------------------
0 | message_passing | BondMessagePassing | 2.2 M  | train
1 | agg             | SumAggregation     | 0      | train
2 | bn              | Identity           | 0      | train
3 | predictor       | RegressionFFN      | 1.7 M  | train
4 | X_d_transform   | Identity           | 0      | train
5 | metrics         | ModuleList         | 0      | train
---------------------------------------------------------------
3.9 M     Trainable params
0         Non-trainable params
3.9 M     Total params
15.487    Total estimated model params size (MB)
28        Modules in train mode
0         Modules in eval mode


Epoch 0: 100%|██████████| 162/162 [00:05<00:00, 30.84it/s, v_num=0, train_loss_step=0.489]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|██████████| 162/162 [00:05<00:00, 28.50it/s, v_num=0, train_loss_step=0.987, val_loss=0.736, train_loss_epoch=0.839]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 2: 100%|██████████| 162/162 [00:05<00:00, 27.39it/s, v_num=0, train_loss_step=0.548, val_loss=0.703, train_loss_epoch=0.732]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 3: 100%|██████████| 162/162 [00:05<00:00, 28.71it/s, v_num=0, train_loss_step=1.110, val_loss=0.695, train_loss_epoch=0.716]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 4: 100%|██████████| 162/162 [00:05<00:00, 28.37it/s, v_num=0, train_loss_step=0.616, val_loss=0.721, train_loss_epoch=0.690]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 5: 100%|██████████| 162/162 [00:05<00:00, 29.08it/s, v_num=0, train_loss_step=0.688, val_loss=0.679, train_loss_epoch=0.712]
Validation: |          |

INFO: You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


...

INFO: You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]



Predicting DataLoader 0: 100%|██████████| 41/41 [00:00<00:00, 54.77it/s]

INFO: Initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/2
INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO: Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/2
INFO: ----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 2 processes
----------------------------------------------------------------------------------------------------

INFO: LOCAL_RANK: 1 - CUDA_VISIBLE_DEVICES: [0,1]
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
INFO: Loading `train_dataloader` to estimate number of stepping batches.


INFO: 
  | Name            | Type               | Params | Mode 
---------------------------------------------------------------
0 | message_passing | BondMessagePassing | 2.2 M  | train
1 | agg             | SumAggregation     | 0      | train
2 | bn              | Identity           | 0      | train
3 | predictor       | RegressionFFN      | 1.7 M  | train
4 | X_d_transform   | Identity           | 0      | train
5 | metrics         | ModuleList         | 0      | train
---------------------------------------------------------------
3.9 M     Trainable params
0         Non-trainable params
3.9 M     Total params
15.487    Total estimated model params size (MB)
28        Modules in train mode
0         Modules in eval mode


Epoch 0: 100%|██████████| 162/162 [00:04<00:00, 35.78it/s, v_num=0, train_loss_step=0.588]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|██████████| 162/162 [00:05<00:00, 27.02it/s, v_num=0, train_loss_step=1.150, val_loss=0.732, train_loss_epoch=0.846]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 2: 100%|██████████| 162/162 [00:05<00:00, 27.53it/s, v_num=0, train_loss_step=0.839, val_loss=0.686, train_loss_epoch=0.774]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 3: 100%|██████████| 162/162 [00:05<00:00, 27.56it/s, v_num=0, train_loss_step=0.512, val_loss=0.704, train_loss_epoch=0.714]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 4: 100%|██████████| 162/162 [00:05<00:00, 27.46it/s, v_num=0, train_loss_step=0.237, val_loss=0.684, train_loss_epoch=0.702]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 5: 100%|██████████| 162/162 [00:05<00:00, 27.33it/s, v_num=0, train_loss_step=0.591, val_loss=0.632, train_loss_epoch=0.668]
Validation: |          |

In [ ]:
predict_command_tr = [
    "chemprop",
    "predict",
    "--test-path", "/kaggle/working/train_clean.csv",
    "--model-paths", '/kaggle/working/models', #model_paths[1], model_paths[2], model_paths[3], model_paths[4], model_paths[5], model_paths[6], model_paths[7], model_paths[8], model_paths[9],
    "--preds-path", f"/kaggle/working/subm_train_1.csv",
    "--smiles-columns", "SMILES",
    "--devices", "1"
]

subprocess.run(predict_command_tr)

2025-04-18T19:22:32 - INFO:chemprop.cli.main - Running in mode 'predict' with args: {'smiles_columns': ['SMILES'], 'reaction_columns': None, 'no_header_row': False, 'num_workers': 0, 'batch_size': 64, 'accelerator': 'auto', 'devices': '1', 'rxn_mode': 'REAC_DIFF', 'multi_hot_atom_featurizer_mode': 'V2', 'keep_h': False, 'add_h': False, 'molecule_featurizers': None, 'descriptors_path': None, 'no_descriptor_scaling': False, 'no_atom_feature_scaling': False, 'no_atom_descriptor_scaling': False, 'no_bond_feature_scaling': False, 'atom_features_path': None, 'atom_descriptors_path': None, 'bond_features_path': None, 'test_path': PosixPath('/kaggle/working/train_clean.csv'), 'output': PosixPath('/kaggle/working/subm_train_1.csv'), 'drop_extra_columns': False, 'model_paths': [PosixPath('/kaggle/working/models')], 'cal_path': None, 'uncertainty_method': 'none', 'calibration_method': None, 'evaluation_methods': None, 'uncertainty_dropout_p': 0.1, 'dropout_sampling_size': 10, 'calibration_interva

Predicting DataLoader 0: 100%|██████████| 203/203 [00:06<00:00, 31.43it/s]


2025-04-18T19:22:41 - INFO:chemprop.cli.predict - Predictions saved to '/kaggle/working/subm_train_1.csv'


CompletedProcess(args=['chemprop', 'predict', '--test-path', '/kaggle/working/train_clean.csv', '--model-paths', '/kaggle/working/models', '--preds-path', '/kaggle/working/subm_train_1.csv', '--smiles-columns', 'SMILES', '--devices', '1'], returncode=0)

In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np

num_models = 1
model_predictions = []
rmse_individual = []

for i in range(num_models):
    subm = pd.read_csv(f'/kaggle/working/subm_train_{i}.csv')
    model_predictions.append(subm['LogP'].values)

all_predictions = np.array(model_predictions).T  

for i in range(1, num_models + 1):
    rmse = np.sqrt(mean_squared_error(train_data['LogP'], all_predictions[:, i]))
    rmse_individual.append(rmse)

mean_prediction = np.mean(all_predictions, axis=1)
rmse_ensemble = np.sqrt(mean_squared_error(train_data['LogP'], mean_prediction))
print(f"\nRMSE для среднего предсказания (ансамбля): {rmse_ensemble}")
print(f"\nМассив RMSE для каждой модели: {rmse_individual}")

In [ ]:
predict_command_test = [
    "chemprop",
    "predict",
    "--test-path", "/kaggle/working/test_clean.csv",
    "--model-paths", '/kaggle/working/model_best', #model_paths[1], model_paths[2], model_paths[3], model_paths[4], model_paths[5], model_paths[6], model_paths[7], model_paths[8], model_paths[9],
    "--preds-path", f"/kaggle/working/subm_test_best.csv",
    "--smiles-columns", "SMILES",
    "--devices", "1"
]

subprocess.run(predict_command_test)

2025-04-18T20:31:01 - INFO:chemprop.cli.main - Running in mode 'predict' with args: {'smiles_columns': ['SMILES'], 'reaction_columns': None, 'no_header_row': False, 'num_workers': 0, 'batch_size': 64, 'accelerator': 'auto', 'devices': '1', 'rxn_mode': 'REAC_DIFF', 'multi_hot_atom_featurizer_mode': 'V2', 'keep_h': False, 'add_h': False, 'ignore_chirality': False, 'molecule_featurizers': None, 'descriptors_path': None, 'no_descriptor_scaling': False, 'no_atom_feature_scaling': False, 'no_atom_descriptor_scaling': False, 'no_bond_feature_scaling': False, 'atom_features_path': None, 'atom_descriptors_path': None, 'bond_features_path': None, 'test_path': PosixPath('/kaggle/working/test_clean.csv'), 'output': PosixPath('/kaggle/working/subm_test_best.csv'), 'drop_extra_columns': False, 'model_paths': [PosixPath('/kaggle/working/model_best')], 'cal_path': None, 'uncertainty_method': 'none', 'calibration_method': None, 'evaluation_methods': None, 'uncertainty_dropout_p': 0.1, 'dropout_sampling

Predicting DataLoader 0: 100%|██████████| 42/42 [00:01<00:00, 27.90it/s]

2025-04-18T20:31:03 - INFO:chemprop.cli.predict - Predictions saved to '/kaggle/working/subm_test_best.csv'


CompletedProcess(args=['chemprop', 'predict', '--test-path', '/kaggle/working/test_clean.csv', '--model-paths', '/kaggle/working/model_best', '--preds-path', '/kaggle/working/subm_test_best.csv', '--smiles-columns', 'SMILES', '--devices', '1'], returncode=0)

In [10]:
import numpy as np
import pandas as pd
pred = pd.read_csv('/kaggle/working/subm_test_best.csv')

subm_example = pd.read_csv('/kaggle/input/submission-example/sibur_element_119_final_sample_submission80.csv', index_col='ID')
subm_example['LogP'] = np.array(pred['LogP'])
subm_example
subm_example.to_csv('/kaggle/working/subm_test_ann1_best.csv')
subm_example


,LogP
ID,
13403,3.716289
13404,4.288357
13405,6.687384
13406,3.296621
13407,7.968871
...,...
16028,3.378249
16029,3.134696
16030,2.548872


In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
from xgboost import XGBRegressor

num_models = 64
model_predictions = []
rmse_individual = []
data_train = pd.read_csv('/kaggle/working/train_clean.csv')

for i in range(num_models):
    subm = pd.read_csv(f'/kaggle/working/subm_test_{i}.csv')
    model_predictions.append(subm['LogP'].values)

all_predictions = np.array(model_predictions).T  # Транспонируем, чтобы строки соответствовали образцам


mean_prediction = np.mean(all_predictions, axis=1)
subm_example = pd.read_csv('/kaggle/input/submission-example/sibur_element_119_final_sample_submission80.csv', index_col='ID')
subm_example['LogP'] = np.array(model_predictions)[34]
subm_example.to_csv('/kaggle/working/subm_test_ann1.csv')


In [126]:
import pandas as pd
from sklearn.metrics import mean_squared_error
import numpy as np

train_clean = pd.read_csv('/kaggle/working/train_clean.csv') # меняй на свой
train_clean['subm'] = train_subm_solo['LogP']
np.sqrt(mean_squared_error(train_clean['LogP'], train_clean['subm']))

0.8797040075228778

In [ ]:
predict_command_tr = [
    "chemprop",
    "predict",
    "--test-path", "/kaggle/working/train_clean.csv",
    "--model-paths", model_paths[0], #model_paths[1], model_paths[2], model_paths[3], model_paths[4], model_paths[5], model_paths[6], model_paths[7], model_paths[8], model_paths[9]...
    "--preds-path", "/kaggle/working/subm_train.csv",
    "--smiles-columns", "SMILES",
    "--devices", "1"
]


subprocess.run(predict_command_tr)

In [ ]:
theory_df = pd.read_csv('/kaggle/input/sibur-happiness/aug_5model.csv')['LogP'] #сверяли MSE с другим датасетом, чтобы не тратить сабмиты   
np.sqrt(mean_squared_error(test_subm_solo['LogP'], theory_df)) 

2.047854806091883

In [ ]:
test_subm_solo.drop('SMILES', axis=1).to_csv('/kaggle/working/subm_test_ann64-2.csv', index=False)